# Imports

In [ ]:
import numpy as np
import sys, os, io
import six.moves.urllib as urllib
import tensorflow as tf
import cv2
import csv

from collections import defaultdict
from io import StringIO
from PIL import Image
from IPython.display import display
import pathlib

# Personnal imports
%run ../utils/object_detection_utils.ipynb
%run ../utils/data_utils.ipynb
%run ../global_variables.ipynb
%run ./detect_variables.ipynb

# Object detection API
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

MODEL_ROOT = os.path.join(OD_ROOT, "ws_fine_tuning/")

# Loading model

In [ ]:
# Model 1
model1_path = os.path.join(OD_ROOT, "ws_fine_tuning/")
model1 = tf.saved_model.load(os.path.join(model1_path,"export/saved_model/"))
model1 = model1.signatures['serving_default']
# Model 2
model2_path = os.path.join(OD_ROOT, "ws_simple_pipeline/")
model2 = tf.saved_model.load(os.path.join(model2_path,"export/saved_model/"))
model2 = model2.signatures['serving_default']
# Category index
PATH_TO_LABELS = os.path.join(MODEL_ROOT,"dataset/binary_label_map.pbtxt")
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Computing stats

In [ ]:
def draw_bbs(source, bbs, bbs_scores=None, min_score=0, colors=None, text_scale=2, title=""):
    font = cv2.FONT_HERSHEY_SIMPLEX
    img = source.copy()
    cv2.putText(img,  title, (70, 150), font, text_scale*4, (255,255,255), 3, cv2.LINE_AA)
    for i, bb in enumerate(bbs):
        color = colors[i]
        if bbs_scores is None or bbs_scores[i]>min_score:
            ymin, xmin, ymax, xmax = bb
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, 4)
            if not bbs_scores is None:
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img,  "{:.2f}".format(bbs_scores[i]), (xmin+5, ymin+int(text_scale*30)), font, text_scale*1, color, 3, cv2.LINE_AA)
    return img

In [ ]:
list_path="/mnt/nvme-storage/pfauregi/training/obj_detection/eval/real_dataset/test_list.csv"
annotations_paths=[]
with open(list_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader:
        annotations_paths.append(''.join(row))

In [ ]:
#annotation_root=os.path.join(DATASET_ROOT, VAL_FOLDER, ANNOTATION_FOLDER)
#annotations_paths = return_all_files_in_folder_rec(annotation_root, ["xml"])
comp_images=True

print("Extracting groundtruth and predicted bounding boxes of "+str(len(annotations_paths))+" images(s).")
a = display(str(0)+"/"+str(len(annotations_paths)),display_id=True)
for i, annotation_path in enumerate(annotations_paths):
    print(annotation_path)
    a.update(str(i+1)+"/"+str(len(annotations_paths)))
    annotation = parse_annotation(annotation_path)
    filename = annotation["filename"].split(".")[0]
    if IMAGE_FOLDER is None:
        print()
        image_path = os.path.join(DATASET_ROOT, VAL_FOLDER, annotation["folder"], annotation["filename"])
    else:
        image_path = os.path.join(DATASET_ROOT, VAL_FOLDER, IMAGE_FOLDER, annotation["filename"])
    image = load_image(image_path, expand=True)
    height, width, channels = image.shape
    # Fetching groundtruth bounding boxes
    groundtruth_bbs = []
    groundtruth_bbs_colors = []
    color_dict = {"diatom":(255, 0, 0), "diatom_floue":(0, 255, 0), "diatom_debri":(0, 0, 255)}
    for obj_bb in annotation["objects"]:
        if obj_bb["name"] in ["diatom", "diatom_floue","diatom_debri"]:
        #if obj_bb["name"] in ["diatom"]:
            groundtruth_bbs.append([obj_bb["ymin"], obj_bb["xmin"], obj_bb["ymax"], obj_bb["xmax"]])
            groundtruth_bbs_colors.append(color_dict[obj_bb["name"]])

    # Model1
    prediction_result = run_inference_for_single_image(model1, image)
    predicted_bbs1 = []
    predicted_bbs_scores1 = []
    predicted_bbs_colors1 = []
    for box, score in zip(prediction_result['detection_boxes'], prediction_result['detection_scores']):
        ymin, xmin, ymax, xmax = int(box[0]*height), int(box[1]*width), int(box[2]*height), int(box[3]*width)
        predicted_bbs1.append([ymin, xmin, ymax, xmax])
        predicted_bbs_scores1.append(score)
        predicted_bbs_colors1.append((255, 0, 0))
        
    # Model2
    prediction_result = run_inference_for_single_image(model2, image)
    predicted_bbs2 = []
    predicted_bbs_scores2 = []
    predicted_bbs_colors2 = []
    for box, score in zip(prediction_result['detection_boxes'], prediction_result['detection_scores']):
        ymin, xmin, ymax, xmax = int(box[0]*height), int(box[1]*width), int(box[2]*height), int(box[3]*width)
        predicted_bbs2.append([ymin, xmin, ymax, xmax])
        predicted_bbs_scores2.append(score)
        predicted_bbs_colors2.append((255, 0, 0))
    
    # Exporting comparison images
    if comp_images:
        image_model1 = draw_bbs(image, predicted_bbs1, colors=predicted_bbs_colors1, bbs_scores=predicted_bbs_scores1, min_score=0.9, text_scale=1, title="Pipeline 1")
        image_model2 = draw_bbs(image, predicted_bbs2, colors=predicted_bbs_colors2, bbs_scores=predicted_bbs_scores2, min_score=0.9, text_scale=1, title="Pipeline 2")
        image_groundtruth = draw_bbs(image, groundtruth_bbs, colors=groundtruth_bbs_colors, text_scale=1, title="Goundtruth")
        # Creating and saving comparison image
        comp_image = np.hstack((image_groundtruth, image_model1, image_model2))
        comp_image_path = os.path.join(OUTPUT_TMP, "images/", filename+"_comp.png")
        save_img(cv2.resize(comp_image, (0,0), fx=0.5, fy=0.5), comp_image_path, compress=5)
        #save_img(image_groundtruth, comp_image_path)
print("Finished!")